In [ ]:
import pandas as pd
pd.set_option('display.max_columns', None)

cleaned_chess_data_path = r'C:\Users\Abrah\Dropbox\PC (2)\Desktop\GitHub Repos\CST499-40_FA22-Capstone-BradleyChess\chess_data\chess_data_cleaned.pkl'
# bradley_q_table_path = r"C:\Users\Abrah\Dropbox\PC (2)\Desktop\GitHub Repos\CST499-40_FA22-Capstone-BradleyChess\Q_Tables\bradley_agent_q_table.pkl"


# kaggle_chess_data = pd.read_pickle(kaggle_chess_data_path, compression = 'zip')
cleaned_chess_data = pd.read_pickle(cleaned_chess_data_path, compression = 'zip')
# bradley_q_table = pd.read_pickle(bradley_q_table_path, compression = 'zip')

In [ ]:
cleaned_chess_data.head(10)

In [ ]:
bradley_q_table.head(20)

In [ ]:
bradley_q_table.shape

In [ ]:
chess_pd_dataframe_file_path_part_1 = r"C:\Users\Abrah\Dropbox\PC (2)\Desktop\Chess_Games_Database\Chess_Games_DB_pd_df_Part_1.pkl"

In [ ]:
chess_data_from_json = pd.read_pickle(chess_pd_dataframe_file_path_part_1, compression = 'zip')

In [ ]:
chess_data_from_json.head(10)

In [ ]:
chess_data_from_json.info(memory_usage='deep')

In [ ]:
games_with_more_than_100_moves = chess_data_from_json[chess_data_from_json['Num Moves'] > 300]

In [ ]:
games_with_more_than_100_moves.head(10)

In [ ]:
games_with_more_than_100_moves.shape

In [ ]:
import pandas as pd
import chess.pgn
import game_settings
import time

def pgn_to_dataframe(input_file_path):
    games_list = []

    with open(input_file_path, 'r', encoding='utf-8') as file:
        while True:
            game = chess.pgn.read_game(file)
            if game is None:
                break

            game_dict = {
                "PlyCount": game.end().board().ply(),
                "Result": game.headers["Result"]
            }

            board = game.board()
            for i, move in enumerate(game.mainline_moves()):
                col_prefix = 'W' if i % 2 == 0 else 'B'
                move_num = i // 2 + 1
                col_name = f"{col_prefix}{move_num}"
                game_dict[col_name] = board.san(move)
                board.push(move)

            games_list.append(game_dict)

    chess_df = pd.DataFrame(games_list)
    chess_df.index = ['Game ' + str(i+1) for i in chess_df.index]
    chess_df = chess_df.fillna('')
    chess_df = chess_df[[c for c in chess_df if c not in ['Result']] + ['Result']]
    chess_df = chess_df[(chess_df['PlyCount'] > 0) & (chess_df['PlyCount'] <= game_settings.max_num_turns_per_player * 2)]

    return chess_df




In [ ]:
start_time = time.time()
df = pgn_to_dataframe(game_settings.chess_pgn_file_path_1)
df.to_pickle(game_settings.chess_pd_dataframe_file_path_part_2, compression='zip')
end_time = time.time()
print('PGN to DataFrame conversion is complete\n')
print(f'It took: {end_time - start_time} seconds')

In [87]:
import game_settings
import pandas as pd
chess_data = pd.read_pickle(game_settings.chess_pd_dataframe_file_path_part_9, compression = 'zip')

In [89]:
chess_data.head()

,PlyCount,W1,B1,W2,B2,W3,B3,W4,B4,W5,B5,W6,B6,W7,B7,W8,B8,W9,B9,W10,B10,W11,B11,W12,B12,W13,B13,W14,B14,W15,B15,W16,B16,W17,B17,W18,B18,W19,B19,W20,B20,W21,B21,W22,B22,W23,B23,W24,B24,W25,B25,W26,B26,W27,B27,W28,B28,W29,B29,W30,B30,W31,B31,W32,B32,W33,B33,W34,B34,W35,B35,W36,B36,W37,B37,W38,B38,W39,B39,W40,B40,W41,B41,W42,B42,W43,B43,W44,B44,W45,B45,W46,B46,W47,B47,W48,B48,W49,B49,W50,B50,W51,B51,W52,B52,W53,B53,W54,B54,W55,B55,W56,B56,W57,B57,W58,B58,W59,B59,W60,B60,W61,B61,W62,B62,W63,B63,W64,B64,W65,B65,W66,B66,W67,B67,W68,B68,W69,B69,W70,B70,W71,B71,W72,B72,W73,B73,W74,B74,W75,B75,W76,B76,W77,B77,W78,B78,W79,B79,W80,B80,W81,B81,W82,B82,W83,B83,W84,B84,W85,B85,W86,B86,W87,B87,W88,B88,W89,B89,W90,B90,W91,B91,W92,B92,W93,B93,W94,B94,W95,B95,W96,B96,W97,B97,W98,B98,W99,B99,W100,B100,W101,B101,W102,B102,W103,B103,W104,B104,W105,B105,W106,B106,W107,B107,W108,B108,W109,B109,W110,B110,W111,B111,W112,B112,W113,B113,W114,B114,W115,B115,W116,B116,W117,B117,W118,B118,W119,B119,W120,B120,W121,B121,W122,B122,W123,B123,W124,B124,W125,B125,W126,B126,W127,B127,W128,B128,W129,B129,W130,B130,W131,B131,W132,B132,W133,B133,W134,B134,W135,B135,W136,B136,W137,B137,W138,B138,W139,B139,W140,B140,W141,B141,W142,B142,W143,B143,W144,B144,W145,B145,W146,B146,W147,B147,W148,B148,W149,B149,W150,B150,W151,B151,W152,B152,W153,B153,W154,B154,W155,B155,W156,B156,W157,B157,W158,B158,W159,B159,W160,B160,W161,B161,W162,B162,W163,B163,W164,B164,W165,B165,W166,B166,W167,B167,W168,B168,W169,B169,W170,B170,W171,B171,W172,B172,W173,B173,W174,B174,W175,B175,W176,B176,W177,B177,W178,B178,W179,B179,W180,Result
Game 1,50,e4,g6,d4,Bg7,f3,d5,e5,c5,c3,Nc6,Be3,cxd4,cxd4,f6,f4,Nh6,Nf3,Bg4,Be2,Nf5,Bf2,fxe5,fxe5,Bxf3,Bxf3,e6,Bg4,O-O,O-O,Bh6,Nc3,Qe7,a3,Ne3,Bxe3,Bxe3+,Kh1,Bxd4,Re1,Bxe5,Rc1,Kh8,b4,Rf2,b5,Qh4,Bh3,Qf4,g3,Rxh2+,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,0-1
Game 2,85,c4,d6,g3,g6,Bg2,Bg7,Nc3,c5,a3,Nc6,Rb1,Bf5,d3,Qd7,h3,h5,Nf3,Rb8,Bd2,a6,b4,cxb4,axb4,b5,cxb5,axb5,Ng5,Nf6,e4,Be6,Nxe6,Qxe6,O-O,Qd7,Ne2,e5,Be3,O-O,f4,d5,f5,d4,Bd2,Kh7,g4,hxg4,hxg4,g5,Bf3,Bh6,Rf2,Ng8,Nc1,f6,Nb3,Rfc8,Nc5,Qf7,Qc1,Nd8,Bd1,Bf8,Bb3,Qg7,Rh2+,Nh6,Qa3,Ra8,Qb2,Bxc5,bxc5,Rxc5,Bb4,Rc7,Kg2,Nf7,Rbh1,Raa7,Kf3,Rc8,Rh5,Rac7,Bxf7,Rxf7,Qh2,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1-0
Game 3,43,c4,Nf6,Nc3,g6,e4,d6,d4,Bg7,f3,O-O,Bg5,c5,d5,h6,Be3,e5,Qd2,Kh7,O-O-O,a6,g4,Ng8,h4,f5,exf5,gxf5,Bd3,Kh8,gxf5,Bxf5,Bxf5,Rxf5,Ne4,b5,Nh3,Rh5,Nhg5,Ra7,Ng3,hxg5,Nxh5,gxh4,Rdg1,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1-0
Game 4,68,c4,c6,d4,d5,Nf3,Nf6,Nc3,dxc4,a4,Na6,e3,Bg4,Bxc4,e6,h3,Bh5,O-O,Nb4,Bd2,Be7,Rc1,O-O,g4,Bg6,Ne5,c5,Nxg6,hxg6,dxc5,Bxc5,Qe2,Qe7,Rfd1,Rfd8,Be1,a6,Qf3,Rac8,Bf1,Ba7,Ne4,Nfd5,g5,Nc6,Kh1,Bb8,Qg2,Be5,b3,Bb2,Rc2,Ba3,Qg3,Bb4,Rc4,Bxe1,Rxe1,Na5,Rxc8,Rxc8,Rb1,Qb4,Qg4,Nc6,Bg2,Ne5,Qd1,Kf8,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,1/2-1/2
Game 5,89,d4,Nf6,c4,e6,Nf3,b6,g3,Ba6,b3,Bb4+,Bd2,Be7,Bg2,c6,Bc3,d5,Nbd2,Nbd7,O-O,O-O,Re1,Rc8,e4,dxe4,Nxe4,c5,Qe2,Bb7,Neg5,cxd4,Nxd4,Bxg2,Kxg2,Nc5,Rad1,Rc7,Kg1,Qc8,Nb5,Rd7,Qc2,R

In [84]:
chess_data.shape

(95058, 353)